# Part 2 - Data Cleaning and Feature Selection

### Contents

<a href = "part-1_EDA.ipynb">Part 1 - Exploratory Data Analysis</a>
    
Part 2 - Data Cleaning

- [Data Cleaning](#Data-Cleaning)
    - [Importing and Basic Data Cleaning](#Importing-and-Basic-Data-Cleaning)
    - [Error Correction](#Error-Correction)
    - [Removing Unnecessary Data and Features](#Removing-Unnecessary-Data-and-Features)
    - [Feature Engineering](#Feature-Engineering)
    - [Creating Lists of Columns for Different Data Types](#Creating-Lists-of-Columns-for-Different-Data-Types)
    - [Filling Null Values](#Filling-Null-Values)
    - [Cleaning Outliers](#Cleaning-Outliers)
    - [Encoding](#Encoding)
    
    
- [Feature Selection](#Feature-Selection)
    - [Feature Selection for Production Model](#Feature-Selection-for_Production-Model)
    - [Feature Selection for Kaggle Model](#Feature-Selection-for-Kaggle-Model)
    
<a href = "part-3_production_model_building.ipynb">Part 3 - Production Model Building</a><br>

#### Importing and Basic Data Cleaning
[top](#Contents)

In [1]:
# imports

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, RFECV

import warnings
warnings.filterwarnings("ignore")

# pandas settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
# importing dataset

train = ("datasets/train.csv")

ames_train = pd.read_csv(train)

In [3]:
# title cleanup

def edit_title (title):
    
    title = (title.replace(" ","_")).lower()
    
    return title

ames_train.rename(columns = lambda i:edit_title(i), inplace = True)

# 3 column names start with numbers. Replacing numbers with strings.

ames_train.rename(columns = {'1st_flr_sf':"first_flr_sf",  '2nd_flr_sf':'second_flr_sf',"3ssn_porch":"threessn_porch"}, inplace = True)

---

## Data Cleaning
[top](#Contents)

---

Several steps were taken to clean the data.

- As mentioned in the EDA, there were several discrepancies with the "year" columns. These columns were further examined, and the following corrections were made:
    - any house with a garage year built before the year built or after the year sold was imputed with the year built and year sold respectively.
    - the same treatment was done to the "year_remod/add" column.
    - finally, any house sold before it was built had the "year_built" value changed to the "yr sold" value.
    - This left one discrepancy, which was manually handled
  
  
- The data dictionary that came with the dataset recommended dropping any house whose "gr_living_area" was above 4000, as they were outliers. This was done.


- 34 columns were removed for either containing no useful information or not enough variance for a model to use for making predictions.


- Of the features remaining, several were features were found to be colinear and removed ('exter_qual', 'kitchen_qual','garage_yr_blt','garage_cars')


- A further examination of the rest of the features revealed that the null value scan be interpreted as a lack of that feature, and thus should be replaced by either "0" or "NA" as required.
    - However, there were discrepancies observed in the bsmt_exposure,bsmt_qual and bsmtfin_type_1 null value counts.
    - A similar discrepancy was also observed in the garage type and garage finish null value count.
    - These discrepancies were imputed with mean or mode values as appropriate

### Error Correction

In [4]:
# A look through the dataset revealed several errors with the columns containing years.

check1 = ames_train[(ames_train["garage_yr_blt"] < ames_train["year_built"]) | (ames_train["garage_yr_blt"] > ames_train["yr_sold"])]
check2 = ames_train[(ames_train["year_remod/add"] < ames_train["year_built"]) | (ames_train["year_remod/add"] > ames_train["yr_sold"]) ]    
check3 = ames_train[(ames_train["year_built"] > ames_train["yr_sold"])] 

# Correcting for errors

for i in check3.index:
    ames_train.at[i, "year_built"] = ames_train.at[i, "yr_sold"]

for i in check1.index:
    ames_train.at[i, "garage_yr_blt"] = ames_train.at[i, "year_built"]
    
for i in check2.index:
    ames_train.at[i, "year_remod/add"] = ames_train.at[i, "year_built"]
    
# manually correcting the remaining row

ames_train.at[1885, "yr_sold"] = 2008

# filling empty garage_yr_blt with year_built
ames_train["garage_yr_blt"].fillna(ames_train["year_built"], inplace = True)

### Removing Unnecessary Data and Features

In [5]:
# as recommended by data dict

outliers = ames_train[ames_train["gr_liv_area"]>4000].index

ames_train.drop(outliers, errors='ignore', axis=0, inplace = True)

In [6]:
no_data = ['id', 'pid','misc_feature', 'misc_val']

l_num = ["mas_vnr_area", "bsmtfin_sf_2","low_qual_fin_sf", "bsmt_half_bath", "half_bath", 
                          "kitchen_abvgr"]

#, "pool_area"

l_nom = ["street","alley","land_contour","condition_1","condition_2","roof_matl","bsmtfin_type_2",
                         "heating","paved_drive","sale_type"]

l_ord = ["utilities", "land_slope","exter_cond","bsmt_cond","central_air", "electrical",
                          "functional", "garage_qual", "garage_cond", "pool_qc"]

low_cor = ["mo_sold","yr_sold"]

high_null = ['fireplace_qu','fence']

comb_list = no_data+l_num+l_nom+l_ord+low_cor+high_null

ames_train.drop(columns = comb_list, inplace = True)

In [7]:
# Removing colinear features

colinear = ['exter_qual', 'kitchen_qual','garage_yr_blt','garage_cars']

ames_train.drop(columns = colinear, inplace = True)



### Feature Engineering

In [8]:
# combining different types of porch sf to a single column

porch_sfs = ['open_porch_sf', 'enclosed_porch', 'threessn_porch','screen_porch']

ames_train['porch_sf'] = ames_train[porch_sfs].sum(axis = 1)

# dropping porch columns

ames_train.drop(columns = porch_sfs, inplace = True)

In [9]:
# log transform year built, year remod and saleprice

def logt(df, col):
    df[col] = np.log(df[col])

for i in ["year_built", "year_remod/add"]:
    logt (ames_train, i)
    
#logt (ames_train, 'saleprice')

### Creating Lists of Columns for Different Data Types

In [10]:
# creating list of continuous data columns

num_cols = [i for i in ames_train.columns if ames_train[i].dtypes == int or ames_train[i].dtypes == float]

# ms_subclass, while numerical in value, is nominal in nature. Removing it from the num_cols list.

# months and years should also be classified as ordinal rather than numerical data

list_nonnums = ["ms_subclass", "year_built", "year_remod/add"]

for i in list_nonnums:
    num_cols.remove(i)
    
# The remanining data had to be cross examined with the data dictionary to determine if it was nominal or ordinal.

# Listing out norminal data

cat_nom_cols = ['ms_subclass','ms_zoning',
 'lot_config',
 'neighborhood',
 'bldg_type',
 'house_style',
 'roof_style',
 'exterior_1st',
 'exterior_2nd',
 'mas_vnr_type',
 'foundation',
 'bsmtfin_type_1',
 'garage_type']

# Creating ordinal data list

cat_ord_cols = [i for i in ames_train.columns if i not in num_cols and i not in cat_nom_cols]

# Changing all num_cols to floats.

for i in num_cols:
    ames_train[i] = ames_train[i].map(lambda x:float(x))

#for the "ms_subclass" column, data should be strings insted of floats

ames_train["ms_subclass"] = ames_train["ms_subclass"].map(lambda x:str(x))

### Filling Null Values

In [11]:
# Showing columns with null values

null_frame = ames_train.isnull().sum().sort_values(ascending=False).to_frame().rename(columns={0:'nulls'})

null_frame[null_frame["nulls"]>0]

,nulls
lot_frontage,330
garage_finish,114
garage_type,113
bsmt_exposure,58
bsmt_qual,55
bsmtfin_type_1,55
mas_vnr_type,22
bsmt_full_bath,2
garage_area,1
bsmtfin_sf_1,1


In [12]:
# cols with < 10 missing value are all numerical. using mean impute.

def mean_impute(df, col):
    for i in col:
        mean = df[i].mean()
        df[i] = df[i].fillna(mean)
    
mean_impute(ames_train, ['total_bsmt_sf', 'garage_area','bsmtfin_sf_1','bsmt_unf_sf','bsmt_full_bath'])

In [13]:
# for the rest of the columns, null values can be interpreted as a lack of that feature.

# however, there are 3 bsmt_exposure null values more than bsmt_qual or bsmtfin_type_1.

bsmt_mis = ames_train[ames_train['bsmt_exposure'].isnull() & ames_train['bsmt_qual'].notnull()]

bsmt_mis

,ms_subclass,ms_zoning,lot_frontage,lot_area,lot_shape,lot_config,neighborhood,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,exterior_1st,exterior_2nd,mas_vnr_type,foundation,bsmt_qual,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmt_unf_sf,total_bsmt_sf,heating_qc,first_flr_sf,second_flr_sf,gr_liv_area,bsmt_full_bath,full_bath,bedroom_abvgr,totrms_abvgrd,fireplaces,garage_type,garage_finish,garage_area,wood_deck_sf,pool_area,saleprice,porch_sf
1456,60,FV,81.0,10411.0,Reg,Corner,Somerst,1Fam,2Story,5.0,5.0,7.604396,7.604396,Gable,VinylSd,VinylSd,None,CBlock,Gd,NaN,Unf,0.0,725.0,725.0,Ex,725.0,863.0,1588.0,0.0,3.0,3.0,8.0,0.0,Attchd,Unf,561.0,0.0,0.0,212109.0,0.0
1547,20,RL,73.0,8987.0,Reg,Inside,Somerst,1Fam,1Story,8.0,5.0,7.603399,7.603898,Gable,VinylSd,VinylSd,BrkFace,PConc,Gd,NaN,Unf,0.0,1595.0,1595.0,Ex,1595.0,0.0,1595.0,0.0,2.0,2.0,6.0,1.0,Attchd,RFn,880.0,144.0,0.0,221500.0,0.0
1997,60,RL,65.0,14006.0,IR1,Inside,CollgCr,1Fam,2Story,7.0,5.0,7.601902,7.601902,Gable,VinylSd,VinylSd,BrkFace,PConc,Gd,NaN,Unf,0.0,936.0,936.0,Ex,936.0,840.0,1776.0,0.0,2.0,3.0,7.0,1.0,Attchd,RFn,474.0,144.0,0.0,192500.0,96.0


In [14]:
for i in bsmt_mis.index:
    ames_train.at[i, "bsmt_exposure"] = ames_train['bsmt_exposure'].mode()[0]

In [15]:
# there is also  garage type and garage finish
garage_mis = ames_train[ames_train['garage_finish'].isnull() & ames_train['garage_type'].notnull()]

garage_mis

,ms_subclass,ms_zoning,lot_frontage,lot_area,lot_shape,lot_config,neighborhood,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,exterior_1st,exterior_2nd,mas_vnr_type,foundation,bsmt_qual,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmt_unf_sf,total_bsmt_sf,heating_qc,first_flr_sf,second_flr_sf,gr_liv_area,bsmt_full_bath,full_bath,bedroom_abvgr,totrms_abvgrd,fireplaces,garage_type,garage_finish,garage_area,wood_deck_sf,pool_area,saleprice,porch_sf
1712,70,RM,50.0,9060.0,Reg,Inside,IDOTRR,1Fam,2Story,5.0,6.0,7.561642,7.600402,Gable,Wd Sdng,Plywood,None,BrkTil,Gd,No,ALQ,548.0,311.0,859.0,Ex,942.0,886.0,1828.0,0.0,2.0,3.0,6.0,0.0,Detchd,NaN,472.878418,174.0,0.0,150909.0,212.0


In [16]:
ames_train.at[garage_mis.index, 'garage_finish'] = ames_train['garage_finish'].mode()[0]

In [17]:
# for the rest of the columns, null values can be interpreted as a lack of that feature.
# filling columns with "NA" or "0" as appropriate

ames_train[cat_nom_cols] = ames_train[cat_nom_cols].fillna("NA")
ames_train[cat_ord_cols] = ames_train[cat_ord_cols].fillna("NA")
ames_train[num_cols] = ames_train[num_cols].fillna(0)

### Cleaning Outliers

In [18]:
# custom function that removes outliers based on IQR limits

def IQR_rule(dataset, column, off = "both"):
    
    error = 0
    
#    p25 = np.percentile(dataset[column], 25)
#    p75 = np.percentile(dataset[column], 75)
    
#    IQR = p75-p25
    
#    botlim = p25 - IQR*1.5
#    toplim = p75 + IQR*1.5
    
    botlim = np.percentile(dataset[column], 1)
    toplim = np.percentile(dataset[column], 99)
    
    if off == "both":
        outliers = dataset[(dataset[column] < botlim) | (dataset[column] > toplim)]
    elif off == "top":
        outliers = dataset[(dataset[column] > toplim)]
    elif off == "bot":
        outliers = dataset[(dataset[column] < botlim)]
    else:
        error = 1
    
    
    if error != 1:
        
        #mean = np.percentile(dataset[column], 50)
        #outliers.apply(lambda i:mean)
        
        #print(f'for the column {column}, {len(outliers)} rows were replaced')
                            
        dataset.drop(outliers.index, errors='ignore', axis=0, inplace = True)
        print(f'for the column {column}, {len(outliers)} rows were dropped')
    
    else:
        print ("error")

In [19]:
#for i in num_cols:
#    IQR_rule(ames_train, i)

IQR_rule(ames_train, 'saleprice')

for the column saleprice, 42 rows were dropped


In [20]:
lot_out = ames_train[ames_train['lot_frontage'] > 300]
lota_out = ames_train[ames_train['lot_area'] > 100000]
wood_out = ames_train[ames_train['wood_deck_sf'] > 1250]
porch_out = ames_train[ames_train['porch_sf'] > 600]

ames_train.drop(lot_out.index, errors='ignore', axis=0, inplace = True)
ames_train.drop(lota_out.index, errors='ignore', axis=0, inplace = True)
ames_train.drop(wood_out.index, errors='ignore', axis=0, inplace = True)
ames_train.drop(porch_out.index, errors='ignore', axis=0, inplace = True)

### Encoding

In [21]:
# lable encoding lot_shape

lot_shape_dict = {"NA":0, "Reg":1, "IR1": 2, "IR2": 3, "IR3":4}

# lable encoding exterqual, extercon, bsmtqual, bsmtcon, heatingqc, kitchenqual, fireplacequ, garagequal, garagecond
# poolqc

qualcon_dict = {"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA":0}

# lable encoding bsmtexposure

bsmtexp_dict = {"Gd":4, "Av":3, "Mn":2, "No":1, "NA":0}

# lable encoding garagefinish

garfin_dict = {"Fin":3, "RFn":2, "Unf":1, "NA":0}

# Combining dictionaries:

combine_dict = {**garfin_dict,**bsmtexp_dict,**qualcon_dict,**lot_shape_dict}


In [22]:
# applying dictionary to ordinal non numerical columns

for i in cat_ord_cols:
    if i not in list_nonnums:
        ames_train[i] = ames_train[i].map(combine_dict)

In [23]:
# converting nominal columns to dummies

dummy_noms = pd.get_dummies(ames_train[cat_nom_cols], drop_first = True)

# dropping nominal columns

ames_train.drop(columns = cat_nom_cols, inplace=True)

In [24]:
final_ames = ames_train.merge(dummy_noms, left_index = True, right_index = True)

---

## Feature Selection

[top](#Contents)

---
The features were selected via incresing the penalty of lasso regression until only 25 features remained. These selected features were cross-checked against a set of 25 features selected by RFE using linear regression, and found to be the same.


A final dataset (final_ames.csv) was generated from these features and the target variable, and saved for model building.

---

In [25]:
# As per convention, splitting varibles into features and target

features = list(final_ames.columns)
features.remove("saleprice")

X = final_ames[features]
y = final_ames["saleprice"]

### Scaling Data

In [26]:
from sklearn.preprocessing import StandardScaler

# Scaling data

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

---

#### Lasso Regression as Feature Selection

In [27]:
# As this is a production model, objective is to cut the number of features down to 30

# to get 30 features, the alpha was iteratively increased until 25 features remained

ls = Lasso(alpha = 0.015)
ls.fit(X_scaled, y)
ls_coef = pd.DataFrame(np.abs(ls.coef_), index = X.columns)
ls_coef.rename(columns = {0:"coef"}, inplace = True)

In [28]:
ls_selection = ls_coef.sort_values("coef" ,ascending = False)
ls_selection = ls_selection[ls_selection["coef"] != 0]

In [29]:
ls_selection.shape

(142, 1)

In [30]:
ls_sel_list = list(ls_selection.index) + ["saleprice"]

In [31]:
final_ames = final_ames[ls_sel_list]

In [32]:
final_ames.to_csv('datasets/final_ames.csv')

#### RFE Feature Selection For Production Model

In [33]:
lr = LinearRegression()

In [34]:
def rfe_list(model, no_of_features):
    rfe = RFE(model, n_features_to_select=no_of_features, verbose =3 )
    rfe.fit(X_scaled,y)
    cols = list(X.columns)
    temp = pd.Series(rfe.support_,index = cols)
    selected_features_rfe = temp[temp==True].index
    return [selected_features_rfe]

In [35]:
rfe_selected = rfe_list(lr, 25)

Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 features.
Fitting estimator with 120 features.
Fitting estimator with 119 features.
Fitting estimator with 118 features.
Fitting estimator with 117 features.
Fitting estimator with 116 features.
F

In [36]:
rfe_list = []

for i in rfe_selected[0]:
    rfe_list.append(i)
    
rfe_list += ["saleprice"]

In [37]:
for i in rfe_list:
    if i not in ls_sel_list:
        print (i)